In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import os
import pickle
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler



In [2]:
def data_collator(data):
    
    return data[0].to(device)

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [4]:
torch.cuda.device_count()

1

In [5]:
if torch.cuda.is_available():
    device = "cuda:0"

In [6]:
model = GPT2LMHeadModel.from_pretrained('gpt2-xl', cache_dir='/bigstor/zsarwar/models/cache/')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl', cache_dir='/bigstor/zsarwar/models/cache/')


In [7]:
model = model.to(device)

In [8]:
infile = "/home/zsarwar/NLP/Sorting-Through-The-Noise/data/combined_data/single_entity_distractor/GPT2XL/small_complete_data_For_MultipleEntityObjectDistractorAccuracyGPT2XL.csv"

In [9]:

def preparedata(infile):
    sentences = []
    labels = []
    
    f = open(infile)
    all_lines = f.readlines()
    for l in all_lines[1:]:
        temp_s = ''.join(l.split('\t')[0]).strip()
        temp_l = ''.join(l.split('\t')[-1]).strip()
        sentences.append(temp_s)
        labels.append(temp_l)
    


    return sentences, labels


In [10]:
sentences = []
labels = []
f = open(infile)
all_lines = f.readlines()
for l in all_lines[1:]:
    temp_s = ''.join(l.split('\t')[0]).strip()
    temp_l = ''.join(l.split('\t')[5]).strip()
    sentences.append(temp_s)
    labels.append(temp_l)
labels = np.asarray(labels)
sentences = np.asarray(sentences)

In [11]:
all_encoded_sentences = []
for s in sentences:
    enc_sent = tokenizer(s, add_special_tokens=False, return_attention_mask=True, return_tensors='pt')
    all_encoded_sentences.append(enc_sent['input_ids'])
labels = np.asarray([l.lower() for l in labels])
all_encoded_sentences = np.asarray(all_encoded_sentences, dtype='object')


/tmp/ipykernel_22780/772034179.py:6: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  all_encoded_sentences = np.asarray(all_encoded_sentences, dtype='object')


In [12]:
encoded_ds = DataLoader(all_encoded_sentences, batch_size=1, sampler=SequentialSampler(all_encoded_sentences), collate_fn=data_collator)

In [13]:
all_predictions = []
#all_scores = []
all_results = []
for idx, prompt in enumerate(encoded_ds):
    if(idx > 500):
        break
    outputs = model.generate(prompt, do_sample=False, max_new_tokens=11, output_scores=True, return_dict_in_generate=True)
    out_scores = outputs.scores
    #all_scores.append(out_scores)
    answer_start_idx = prompt.size()[1]
    gen_answer = tokenizer.decode(outputs.sequences[0][answer_start_idx:], skip_special_tokens=True)
    all_predictions.append(gen_answer)
    # Compute answer
    if(labels[idx] in gen_answer.lower()):
        all_results.append('Correct')
    else:
        all_results.append('Incorrect')
    


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [14]:
all_predictions = np.asarray(all_predictions)

In [15]:
all_results = np.asarray(all_results)

In [16]:
all_results

array(['Correct', 'Correct', 'Correct', 'Correct', 'Correct', 'Correct',
       'Correct', 'Correct', 'Correct', 'Correct', 'Correct', 'Correct',
       'Correct', 'Correct', 'Correct', 'Correct', 'Correct', 'Correct',
       'Correct', 'Correct', 'Incorrect', 'Correct', 'Correct', 'Correct',
       'Correct', 'Correct', 'Correct', 'Correct', 'Correct', 'Correct',
       'Correct', 'Correct', 'Correct', 'Correct', 'Correct', 'Correct',
       'Correct', 'Correct', 'Correct', 'Correct', 'Correct', 'Correct',
       'Correct', 'Correct', 'Correct', 'Correct', 'Correct', 'Correct',
       'Correct', 'Correct', 'Correct', 'Correct', 'Correct', 'Correct',
       'Correct', 'Correct', 'Correct', 'Correct', 'Correct', 'Correct',
       'Correct', 'Correct', 'Correct', 'Correct', 'Correct', 'Correct',
       'Correct', 'Correct', 'Correct', 'Correct', 'Correct', 'Correct',
       'Correct', 'Correct', 'Correct', 'Correct', 'Correct', 'Correct',
       'Correct', 'Correct', 'Correct', 'Correct'

In [17]:
incorrect_indices = np.where(all_results == 'Incorrect')

In [20]:
len(incorrect_indices[0])

100

In [23]:
id = 85
sentences[id]

"Jake has a sister , played basketball and now lives in France and has travelled to Beijing , Warsaw and Santiago . The capital of Jake's country is"

In [24]:
all_predictions[id]

" Paris. Jake's favourite food is French fries. Jake"

In [25]:
labels[id]

'paris'

In [26]:
incorrect_indices

(array([ 20,  87, 115, 116, 128, 129, 155, 156, 157, 158, 168, 169, 170,
        184, 195, 196, 197, 199, 200, 207, 208, 209, 210, 211, 212, 215,
        216, 219, 220, 221, 222, 223, 224, 227, 228, 229, 230, 231, 235,
        236, 237, 238, 241, 242, 257, 260, 261, 262, 296, 297, 298, 344,
        345, 346, 349, 352, 353, 356, 357, 358, 360, 361, 368, 369, 370,
        372, 376, 377, 380, 432, 436, 437, 438, 440, 441, 442, 444, 445,
        446, 448, 449, 450, 452, 453, 454, 456, 457, 460, 461, 462, 464,
        465, 466, 468, 469, 470, 472, 476, 477, 478]),)

In [27]:
sentences[incorrect_indices]

array(["Rowan has a sister and now lives in France and has travelled to Santiago , Helsinki and Warsaw . The capital of Rowan's country is",
       "Rowan lives in France and has travelled to Beijing , Warsaw and Helsinki . The capital of Rowan's country is",
       "Rowan lives in France and has travelled to Helsinki , Beijing and Warsaw . The capital of Rowan's country is",
       "Rowan has a sister and now lives in France and has travelled to Helsinki , Beijing and Warsaw . The capital of Rowan's country is",
       "Sebastian has a sister and now lives in France and has travelled to Helsinki , Jakarta and Warsaw . The capital of Sebastian's country is",
       "Sebastian has a sister , played basketball and now lives in France and has travelled to Helsinki , Jakarta and Warsaw . The capital of Sebastian's country is",
       "Rowan lives in France and has travelled to Jakarta , Beijing and Santiago . The capital of Rowan's country is",
       "Rowan has a sister and now lives in F

In [28]:
all_sents_incorrect = sentences[incorrect_indices]

In [29]:
all_preds_incorrect = all_predictions[incorrect_indices]

In [30]:
all_labels_incorrect = labels[incorrect_indices]

In [31]:
out_dir = "/home/zsarwar/NLP/NLP_Repo/NLP_Project/Results/GPT2-XL/"

In [32]:
sents_preds_labels = {}
sents_preds_labels['sentences'] = all_sents_incorrect
sents_preds_labels['predictions'] = all_preds_incorrect
sents_preds_labels['labels'] = all_labels_incorrect

In [33]:
out_file = infile.split('/')[-1].split('.')[0]
out_file = out_dir + "Incorrect_predictions_" + out_file + ".pickle"
out_file

'/home/zsarwar/NLP/NLP_Repo/NLP_Project/Results/GPT2-XL/Incorrect_predictions_small_complete_data_For_MultipleEntityObjectDistractorAccuracyGPT2XL.pickle'

In [34]:
with open(out_file, 'wb') as out_handle:
    pickle.dump(sents_preds_labels, out_handle)

In [35]:
# Loading a pickle object
with open(out_file, 'rb') as in_file:
    s_l = pickle.load(in_file)


In [36]:
s_l['sentences']

array(["Rowan has a sister and now lives in France and has travelled to Santiago , Helsinki and Warsaw . The capital of Rowan's country is",
       "Rowan lives in France and has travelled to Beijing , Warsaw and Helsinki . The capital of Rowan's country is",
       "Rowan lives in France and has travelled to Helsinki , Beijing and Warsaw . The capital of Rowan's country is",
       "Rowan has a sister and now lives in France and has travelled to Helsinki , Beijing and Warsaw . The capital of Rowan's country is",
       "Sebastian has a sister and now lives in France and has travelled to Helsinki , Jakarta and Warsaw . The capital of Sebastian's country is",
       "Sebastian has a sister , played basketball and now lives in France and has travelled to Helsinki , Jakarta and Warsaw . The capital of Sebastian's country is",
       "Rowan lives in France and has travelled to Jakarta , Beijing and Santiago . The capital of Rowan's country is",
       "Rowan has a sister and now lives in F

In [37]:
s_l['predictions']

array([' Warsaw.\n\nRowan has a sister and now',
       " not known.\n\nRowan's mother, who",
       " not known.\n\nRowan's mother, who",
       " not known.\n\nRowan's mother is a",
       ' Warsaw.\n\nSebastian is a member',
       ' Warsaw, where he is a student at the University of',
       " Jakarta.\n\nRowan's first book, The",
       ' Jakarta.\n\nRowan has a sister and now',
       ' Jakarta.\n\nRowan has a sister, played',
       ' Jakarta.\n\nRowan has a sister, played',
       ' Jakarta.\n\nRowan has a sister and now',
       ' Jakarta.\n\nRowan has a brother, played',
       ' Jakarta.\n\nRowan has a sister, played',
       " Jakarta.\n\nRowan's father, who is",
       ' Warsaw.\n\nHe is a member of the International',
       ' Warsaw.\n\nJoe has a sister and now lives',
       ' Warsaw, where he has a friend. Joe has a',
       ' Warsaw.\n\nHe is a member of the Polish',
       ' Warsaw.\n\nJake is a fan of the band',
       " Warsaw.\n\nRowan's first book, The",
       '

In [38]:
s_l['labels']

array(['paris', 'paris', 'paris', 'paris', 'paris', 'paris', 'paris',
       'paris', 'paris', 'paris', 'paris', 'paris', 'paris', 'paris',
       'paris', 'paris', 'paris', 'paris', 'paris', 'paris', 'paris',
       'paris', 'paris', 'paris', 'paris', 'paris', 'paris', 'paris',
       'paris', 'paris', 'paris', 'paris', 'paris', 'paris', 'paris',
       'paris', 'paris', 'paris', 'paris', 'paris', 'paris', 'paris',
       'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakarta',
       'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakarta',
       'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakarta',
       'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakarta',
       'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakarta',
       'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakarta',
       'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakarta',
       'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakarta', 'jakart